## Ask A Manager Salary Survey 2021 (Responses).

[This dataset I've found on internet](https://docs.google.com/spreadsheets/d/1IPS5dBSGtwYVbjsfbaMCYIWnOuRmJcbequohNxCyGVw/edit?usp=sharing) represents the responses of managers from different countries of the world. to my knowledge, the dataset is 'live', which means it's constantly growing. The survey itself can be filled out [here](https://www.askamanager.org/2021/04/how-much-money-do-you-make-4.html)

### Let's clean it!

In [28]:
#importing my 'everyday' tools to work with dataset
import pandas as pd
import matplotlib.pyplot as plt
from adjustText import adjust_text
import seaborn as sns
%matplotlib inline

In [29]:
dataset_main = pd.read_csv('ask_a_manager.csv', parse_dates = ['Timestamp'])
copy_df = dataset_main.copy()

In [30]:
copy_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27620 entries, 0 to 27619
Data columns (total 18 columns):
 #   Column                                    Non-Null Count  Dtype         
---  ------                                    --------------  -----         
 0   Timestamp                                 27620 non-null  datetime64[ns]
 1   How old are you?                          27620 non-null  object        
 2   Industry                                  27551 non-null  object        
 3   Job title                                 27620 non-null  object        
 4   Additional context on job title           7151 non-null   object        
 5   Annual salary                             27620 non-null  object        
 6   Other monetary comp                       20472 non-null  float64       
 7   Currency                                  27620 non-null  object        
 8   Currency - other                          182 non-null    object        
 9   Additional context on income

In [31]:
#removing duplicates if present
copy_df = copy_df.drop_duplicates()

In [32]:
# Trimming whitespaces, extra characters in the whole dataset in advance
df_object_types = copy_df.select_dtypes(['object']) #selecting columns with 'object', string type
copy_df[df_object_types.columns] = df_object_types.apply(lambda x: x.str.strip()) #trimming columns

In [33]:
# lowering characters in column headers to maintain consistency 
copy_df.columns = [x.lower() for x in copy_df.columns]

In [34]:
copy_df['timestamp'] = copy_df['timestamp'].dt.normalize() #normalizing 'timestamp' column to show the date without time

In [35]:
# Converting 'Annual Salary' columns into int type
copy_df['annual salary'] = copy_df['annual salary'].replace(',','', regex=True).astype(int)

In [36]:
copy_df.head()

,timestamp,how old are you?,industry,job title,additional context on job title,annual salary,other monetary comp,currency,currency - other,additional context on income,country,state,city,overall years of professional experience,years of experience in field,highest level of education completed,gender,race
0,2021-04-27,25-34,Education (Higher Education),Research and Instruction Librarian,NaN,55000,0.0,USD,NaN,NaN,United States,Massachusetts,Boston,5-7 years,5-7 years,Master's degree,Woman,White
1,2021-04-27,25-34,Computing or Tech,Change & Internal Communications Manager,NaN,54600,4000.0,GBP,NaN,NaN,United Kingdom,NaN,Cambridge,8 - 10 years,5-7 years,College degree,Non-binary,White
2,2021-04-27,25-34,"Accounting, Banking & Finance",Marketing Specialist,NaN,34000,NaN,USD,NaN,NaN,US,Tennessee,Chattanooga,2 - 4 years,2 - 4 years,College degree,Woman,White
3,2021-04-27,25-34,Nonprofits,Program Manager,NaN,62000,3000.0,USD,NaN,NaN,USA,Wisconsin,Milwaukee,8 - 10 years,5-7 years,College degree,Woman,White
4,2021-04-27,25-34,"Accounting, Banking & Finance",Accounting Manager,NaN,60000,7000.0,USD,NaN,NaN,US,South Carolina,Greenville,8 - 10 years,5-7 years,College degree,Woman,White


# cleaning the 'country' column.
Time to clean "country" column, which seems to be severely inconsistent

In [37]:
copy_df["country"].unique()

array(['United States', 'United Kingdom', 'US', 'USA', 'Canada', 'usa',
       'UK', 'Scotland', 'U.S.', 'The Netherlands', 'Australia', 'Spain',
       'us', 'Usa', 'England', 'finland', 'United States of America',
       'France', 'United states', 'Germany', 'united states', 'Ireland',
       'India', 'Uk', 'canada', 'U.S>', 'ISA', 'Argentina',
       'Great Britain', 'United State', 'U.S.A', 'Denmark', 'U.S.A.',
       'America', 'Netherlands', 'netherlands',
       'united states of america', 'Switzerland', 'Bermuda', 'Us',
       'The United States', 'United State of America', 'Malaysia',
       'Mexico', 'United Stated', 'South Africa', 'Belgium',
       'Northern Ireland', 'u.s.', 'UNITED STATES', 'united States',
       'Sweden', 'Hong Kong', 'Kuwait', 'Norway', 'Sri lanka',
       'Contracts', 'USA-- Virgin Islands', 'United Statws', 'England/UK',
       'U.S',
       "We don't get raises, we get quarterly bonuses, but they periodically asses income in the area you work, so I 

There are plenty of variants of the same countries present. The most inconsistent one is USA. My decision here is to separate 'USA' variants from others first, finding all occurrences and replacing them with one - 'US'.

In [38]:
US_list = ['United States', 
 'USA', 'usa', 'U.S.','us', 'Usa',
 'United States of America', 
 'United states',  'united states','U.S>', 'ISA','United State', 
 'U.S.A', 'U.S.A.','America', 'united states of america', 'Us',
 'The United States', 'United State of America', 'United Stated','u.s.', 'UNITED STATES', 'united States',
'USA-- Virgin Islands', 'United Statws', 'U.S', 'Unites States', 'U. S.','United Sates',  
 'United States of American', 'Uniited States', 'United Sates of America',
'Unted States', 'United Statesp', 'United States (I work from home and my clients are all over the US/Canada/PR',
 'United Stattes', 'United Statea', 'United Statees', 'UNited States','Uniyed states', 'Uniyes States', 
 'United States of Americas','U.A.', 'US of A', 'United States of america','U.SA',
 'United Status', 'Virginia', 'U.s.', 'U.s.a.', 'USS', 'Uniteed States', 'US Gov position','United Stares', 
 'I.S.', 'Unites states', 'Unite States', 'The US', 'United states of America',
 'For the United States government, but posted overseas', 'IS', 'UnitedStates', 'United statew',
'bonus based on meeting yearly goals set w/ my supervisor', 
 "I earn commission on sales. If I meet quota, I'm guaranteed another 16k min. Last year i earned an additional 27k. It's not uncommon for people in my space to earn 100k+ after commission.",
 'United Statues', 'Untied States','USA (company is based in a US territory, I work remote)','USAB',
'Unitied States','United Sttes', 'united stated','United States Of America',
'I was brought in on this salary to help with the EHR and very quickly was promoted to current position but compensation was not altered.',
'Uniter Statez', 'U. S', 'USA tomorrow', 'United Stateds', 'UXZ',
'US govt employee overseas, country withheld', 'San Francisco', 'Usat',
'🇺🇸', 'Unitef Stated', 'United STates', 'USaa', 'uSA', 'america', 'Hartford',
'Y', 'United y', 'California','uS', 'USD', "USA, but for foreign gov't", 'Worldwide (based in US but short term trips aroudn the world)',
'United Statss', 'United states of america', 'UsA','I work for a UAE-based organization, though I am personally in the US.',
'United  States', 'Aotearoa New Zealand', 'na','United States is America',
"We don't get raises, we get quarterly bonuses, but they periodically asses income in the area you work, so I got a raise because a 3rd party assessment showed I was paid too little for the area we were located"]

Unfortunately, responders have decided to make my life way harder than it should be. Besides basic typo occurrences there are answers which were not supposed to be in this column in the first place. By extracting rows based on these responses, I've found that these non-relevant answers in 'country' column look like they belong to 'additional context on income'. I believe moving these instances into the second column is a reasonable decision.

In [39]:
# Assigning values from 'country' to 'additional context on income'
copy_df.at[16853, 'additional context on income'] = 'I was brought in on this salary to help with the EHR and very quickly was promoted to current position but compensation was not altered.'
copy_df.at[12925, 'additional context on income'] =  "I earn commission on sales. If I meet quota, I'm guaranteed another 16k min. Last year i earned an additional 27k. It's not uncommon for people in my space to earn 100k+ after commission."
copy_df.at[12504, 'additional context on income'] = 'bonus based on meeting yearly goals set w/ my supervisor'
copy_df.at[2249, 'additional context on income'] = "We don't get raises, we get quarterly bonuses, but they periodically asses income in the area you work, so I got a raise because a 3rd party assessment showed I was paid too little for the area we were located"

In [40]:
# replacing all inconsistent values of 'US'
copy_df = copy_df.replace(US_list, 'US')

'UK' values correction

In [41]:
UK_list = ['United Kingdom', 'UK', 'Scotland',
        'England',  'Uk', 'canada', 'Great Britain', 'Northern Ireland', 'Contracts',
       'England/UK', 'England, UK.', 'Britain',
        'Global', 'United Kingdom (England)',
        'ireland', 'United Kingdom.',
       'United kingdom', 'U.K.', 
       'United Kindom', 'Currently finance', 'England, UK',
        'uk', 'UK (Northern Ireland)',
       'UK for U.S. company', 'United Kingdomk', 
       'united kingdom', 'Wales (United Kingdom)', 'England, Gb', 'U.K. (northern England)', 'U.K', 
       'England, United Kingdom', 'Englang',
       'Wales', 'UK (England)', 'UK, remote', 'Scotland, UK', 'Unites kingdom', 
        'england', 'Wales, UK', 'Wales (UK)',
        'Northern Ireland, United Kingdom',
       'UK, but for globally fully remote company',
       'ENGLAND', 'Ireland', 'UK/UK', 'UK, UK.', 'UK (UK)', 'UK.',  'UK, UK','UKk',
          'UK (northern UK)', 'UK, Gb', 'UK (northern UK)' ]

In [42]:
copy_df = copy_df.replace(UK_list, 'UK')

CANADA!

In [43]:
CAN_list = ['Canada', 'Canada, Ottawa, ontario',
       'CANADA', 'Canadw', 'Can',
       'I am located in Canada but I work for a company in the US',
       'Canda', 'Canada and USA',
       '$2,175.84/year is deducted for benefits', 'Csnada',
        'Canad', 'Remote', 'Singapore', 'Canadá', 'Policy']

In [44]:
# Additional context on income is in the wrong column. There goes the fix
copy_df.at[8383, 'additional context on income'] = '$2,175.84/year is deducted for benefits'

In [45]:
copy_df = copy_df.replace(CAN_list, 'Canada')

creating lists of remaining countries

In [46]:
CZ_list = ['Czech republic', 'Czechia', 'czech republic', 'Czech Republic', 'europe']
netherlands_list = ['The Netherlands', 'Netherlands', 'netherlands', 'the Netherlands', 'Nederland',
                   'The netherlands', 'the netherlands', 'NL']
NZ_list = ['New Zealand', 'New Zealand Aotearoa', 'New zealand', 'NZ', 'new zealand',
          'From New Zealand but on projects across APAC']
GER_list = ['Germany', 'germany']
FR_list = ['France', 'FRANCE', 'france']
FIN_list = ['finland', 'Finland' ]
JPN_list = ['Japan', 'japan', 'Japan, US Gov position']
UAE_list = ['United Arab Emirates','UAE']
LUX_list = ['Luxembourg', 'Luxemburg']
SPAIN_list = ['Spain','spain' ]
UA_list = ['UA', 'Ukraine']
CROATIA_list = ['Croatia', 'croatia']
ITL_list = ['Italy (South)', 'Italy']
HK_list = ['Hong Kong','hong konh']
AUS_list = ['Australia', 'australia', 'Australi', 'Australian']
CN_list = ['Mainland China', ' China']
CH_list = ['switzerland', 'Switzerland', 'SWITZERLAND'] 
AT_list = ['Austria', 'Austria, but I work remotely for a Dutch/British company']
MX_list = ['Mexico','México' ]
DK_list = ['Danmark', 'denmark' ]
SA_list = ['South Africa', 'South africa']
PH_list = ['Philippines', 'Remote (philippines)']
ARG_list = ['ARGENTINA BUT MY ORG IS IN THAILAND', "I work for an US based company but I'm from Argentina."]
na_list = ['International', 'n/a (remote from wherever I want)']


Replacing the values and maintaining consistency...

In [47]:
copy_df = copy_df.replace(CZ_list, 'Czech Republic')
copy_df = copy_df.replace(netherlands_list, 'Netherlands')
copy_df = copy_df.replace(NZ_list, 'New Zealand')
copy_df = copy_df.replace(GER_list, 'Germany')
copy_df = copy_df.replace(FR_list, 'France')
copy_df = copy_df.replace(FIN_list, 'Finland')
copy_df = copy_df.replace(JPN_list, 'Japan')
copy_df = copy_df.replace(UAE_list, 'UAE')
copy_df = copy_df.replace(LUX_list, 'Luxembourg')
copy_df = copy_df.replace(SPAIN_list, 'Spain')
copy_df = copy_df.replace(CROATIA_list, 'Croatia')
copy_df = copy_df.replace(ITL_list, 'Italy')
copy_df = copy_df.replace(HK_list, 'Hong Kong')
copy_df = copy_df.replace(AUS_list, 'Australia')
copy_df = copy_df.replace(CN_list, 'China')
copy_df = copy_df.replace('singapore', 'Singapore', regex = True)
copy_df = copy_df.replace(CH_list, 'Switzerland')
copy_df = copy_df.replace('From Romania, but for an US based company', 'Romania')
copy_df = copy_df.replace(AT_list, 'Austria')
copy_df = copy_df.replace(MX_list, 'Mexico')
copy_df = copy_df.replace(DK_list, 'Denmark')
copy_df = copy_df.replace(SA_list, 'South Africa')
copy_df = copy_df.replace(UA_list, 'Ukraine')
copy_df = copy_df.replace('United States- Puerto Rico', 'Puerto Rico')
copy_df = copy_df.replace(PH_list, 'Philippines')
copy_df = copy_df.replace('Company in Germany. I work from Pakistan.', 'Pakistan')
copy_df = copy_df.replace(ARG_list, 'Argentina')
copy_df = copy_df.replace(na_list, 'Global')

In [48]:
copy_df['country'].unique()

array(['US', 'UK', 'Canada', 'Netherlands', 'Australia', 'Spain',
       'Finland', 'France', 'Germany', 'India', 'Argentina', 'Denmark',
       'Switzerland', 'Bermuda', 'Malaysia', 'Mexico', 'South Africa',
       'Belgium', 'Sweden', 'Hong Kong', 'Kuwait', 'Norway', 'Sri lanka',
       'Greece', 'Japan', 'Austria', 'Brazil', 'Hungary', 'Luxembourg',
       'Colombia', 'New Zealand', 'Trinidad and Tobago', 'Cayman Islands',
       'Czech Republic', 'Latvia', 'Puerto Rico', 'Rwanda', 'UAE',
       'Bangladesh', 'Romania', 'Serbia', 'Philippines', 'Russia',
       'Poland', 'Turkey', 'Catalonia', 'Italy',
       'Jersey, Channel islands', 'China', 'Afghanistan', 'Israel',
       'Taiwan', 'Cambodia', 'Vietnam', 'South Korea', 'Thailand',
       'Lithuania', 'Eritrea', 'Indonesia', 'Cuba', 'Slovenia',
       "Cote d'Ivoire", 'Somalia', 'Sri Lanka', 'Slovakia', 'Portugal',
       'Sierra Leone', 'Global', 'The Bahamas', 'Costa Rica', 'Chile',
       'Qatar', 'Nigeria', 'Panamá', 'Congo',

# Part II - Optimizing the remaining columns



### 2.1. Too many unique states

There are 50 states that form the USA (wow!). It's apparent that respondends to have some fun with answer options, clicking not one or two, but simultaneously up to 5 states. I've decided that 'state' column should consist of states that were clicked on/chosen first.

In [49]:
# split the whole string into the list, extracting the first state from it, trimming whitespaces and extra char-s
copy_df['state'] = copy_df['state'].str.split(', ').str.get(0).str.strip()

In [50]:
# 50 states + 1 nan
copy_df['state'].nunique()

51

### 2.2. Too many race variants is selected by one respondent 

'Race' column has the similar problem, and it's going to be handled the same way.

In [51]:
copy_df['race'] = copy_df['race'].str.split(', ').str.get(0).str.strip()

### 2.3. Inconsistencies in 'cities' column

In [52]:
copy_df['city'].nunique()

4157

In [72]:
copy_df['city'] = copy_df['city'].str.title() 
copy_df['city'] = copy_df['city'].str.split(', ').str.get(0).str.strip()
copy_df['city'] = copy_df["city"].replace(['New York', 'Nyc (Remote)', 'Nyc'], 'New York City' )


In [73]:
copy_df['city'].nunique()

3477

I'll be back to improve this part as soon as I'll have required experience in dealing with these kinds of problems. For now, I reduced the size of unique values from 4157 to 3477.

### 2.4. Currency trouble

I noticed that many respondents from neither Australia nor New Zealand are happy about their currency options. I'll handle it buy splitting AUS/NZD, assigning them to their respective countries.

In [119]:
#reorganizing the currency values...
copy_df.loc[copy_df['country'] == "New Zealand", "currency"] = "NZD"
copy_df.loc[copy_df['country'] == "Australia", "currency"] = "AUD"

#...simultaneously deleting complaints about them
copy_df.loc[copy_df['country'] == "Australia", "currency - other"] = "" 
copy_df.loc[copy_df['country'] == "New Zealand", "currency - other"] = ""

In [138]:
# Treating US the same way
copy_df.loc[copy_df['country'] == "US", "currency"] = "USD" #USD was labeled as US for some reason. 
copy_df.loc[copy_df['country'] == "US", "currency - other"] = "" #there were no people other than 
                                                                 #US respondents who use 'US' as a currency 
                                                                 #in the secondary column 
copy_df.at[[25510,26236], 'currency'] = 'USD'

In [144]:
# and Canada, too
copy_df.loc[copy_df['country'] == "Canada", "currency"] = "CAD"
copy_df.loc[copy_df['country'] == "Canada", "currency - other"] = ""
copy_df.at[17426, 'currency'] = 'CAD'

In [253]:
#treating some individual examples
copy_df.at[3261, 'currency'] = 'EUR'
copy_df.loc[copy_df['country'] == "Hungary", "currency - other"] = "" #respondent from Hungary
copy_df.loc[copy_df['country'] == "Czech Republic", "currency - other"] = "CZK" #Czech Republic 
copy_df.at[3145, 'country'] = 'Canada'
copy_df.loc[copy_df['country'] == "Canada", "currency - other"] = "" #Canada
copy_df.at[[5396,5871] , 'currency'] = 'GBP'
copy_df.loc[copy_df['country'] == "UK", "currency - other"] = "" #UK
copy_df.loc[copy_df['country'] == "Philippines", "currency - other"] = "PHP" #Phillippines
copy_df.loc[copy_df['country'] == "Brazil", "currency - other"] = "BRL"
copy_df.loc[copy_df['country'] == "Brazil", "currency"] = ""
copy_df.loc[copy_df['country'] == "Denmark", "currency - other"] = "DKK"
copy_df.loc[copy_df['country'] == "Denmark", "currency"] = ""
copy_df.loc[copy_df['country'] == "Argentina", "currency - other"] = "ARS"
copy_df.loc[copy_df['country'] == "Mexico", "currency - other"] = "MXN"
copy_df.loc[copy_df['country'] == "India", "currency - other"] = "INR"
copy_df.loc[copy_df['country'] == "Bangladesh", "currency - other"] = "BDT"
copy_df.loc[copy_df['country'] == "Poland", "currency - other"] = "PLN"
copy_df.loc[copy_df['country'] == "Norway", "currency - other"] = "NOK"
copy_df.loc[copy_df['country'] == "Israel", "currency - other"] = "ILS/NIS"
copy_df.loc[copy_df['country'] == "China", "currency - other"] = "RMB"
copy_df.loc[copy_df['country'] == "South Korea", "currency - other"] = "KRW"
copy_df.loc[copy_df['country'] == "Thailand", "currency - other"] = "THB"
copy_df.loc[copy_df['country'] == "Croatia", "currency - other"] = "KN"
copy_df.loc[copy_df['country'] == "Taiwan", "currency - other"] = "NTD"
copy_df.loc[copy_df['country'] == "Pakistan", "currency - other"] = ""

In [254]:
copy_df['currency'].unique()

array(['USD', 'GBP', 'CAD', 'EUR', 'AUD', 'Other', '', 'CHF', 'ZAR',
       'SEK', 'HKD', 'NZD', 'JPY'], dtype=object)

In [255]:
copy_df['currency - other'].unique()

array(['', nan, 'INR', 'ARS', 'DKK', 'MYR', 'MXN', 'CHF', 'KWD', 'NOK',
       'BRL', 'SEK', 'COP', 'TTD', 'CZK', 'BDT', 'ZAR', 'PHP', 'PLN',
       'TRY', 'RMB', 'ILS/NIS', 'NTD', 'KRW', 'THB', 'IDR', 'LKR', 'SAR',
       'RM', 'KN'], dtype=object)

In [256]:
copy_df

,timestamp,how old are you?,industry,job title,additional context on job title,annual salary,other monetary comp,currency,currency - other,additional context on income,country,state,city,overall years of professional experience,years of experience in field,highest level of education completed,gender,race
0,2021-04-27,25-34,Education (Higher Education),Research and Instruction Librarian,NaN,55000,0.0,USD,,NaN,US,Massachusetts,Boston,5-7 years,5-7 years,Master's degree,Woman,White
1,2021-04-27,25-34,Computing or Tech,Change & Internal Communications Manager,NaN,54600,4000.0,GBP,,NaN,UK,NaN,Cambridge,8 - 10 years,5-7 years,College degree,Non-binary,White
2,2021-04-27,25-34,"Accounting, Banking & Finance",Marketing Specialist,NaN,34000,NaN,USD,,NaN,US,Tennessee,Chattanooga,2 - 4 years,2 - 4 years,College degree,Woman,White
3,2021-04-27,25-34,Nonprofits,Program Manager,NaN,62000,3000.0,USD,,NaN,US,Wisconsin,Milwaukee,8 - 10 years,5-7 years,College degree,Woman,White
4,2021-04-27,25-34,"Accounting, Banking & Finance",Accounting Manager,NaN,60000,7000.0,USD,,NaN,US,South Carolina,Greenville,8 - 10 years,5-7 years,College degree,Woman,White
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27615,2022-01-17,25-34,Sales,Sales Representative,NaN,43000,500.0,USD,,NaN,US,Minnesota,Minneapolis,1 year or less,1 year or less,College degree,Woman,White
27616,2022-01-18,25-34,Engineering or Manufacturing,Staff Engineer,Local Gov. City Planning,76000,5000.0,USD,,NaN,US,North Dakota,Williston,5-7 years,1 year or less,College degree,Woman,White
27617,2022-01-18,25-34,Engineering or Manufacturing,Petroleum Engineer,field engineer,96000,45000.0,USD,,NaN,US,North Dakota,Williston,5-7 years,5-7 years,College degree,Woman,White
27618,2022-01-20,35-44,Computing or Tech,Head of Business Operations,NaN,210000,40000.0,USD,,NaN,US,US,Los Angeles,11 - 20 years,2 - 4 years,Master's degree,Man,Another option not listed here or prefer not t...
